In [21]:
import numpy as np

# функция для расчета стоимости решения
def cost(T, D, n):
    L = 0
    for i in range(n):
        if i == n-1:
            L += D[T[i]][T[0]]
        else:
            L += D[T[i]][T[i+1]]
    return L
#-------------------------------------------------------------------------------------------------
def peak_search(m, a, b, tay, D, i):
    # матрица с вероятностями
    P = np.zeros(len(m), dtype=float)
    # Pz - знаменатель вероятности перехода
    Pz = 0
    # вершины для которых вычисляеться вероятность
    v = []
    # находим знаменатель вероятности перехода для данного муравья
    for j, val in enumerate(m):
        if val == 0:
            Pz += (tay[i][j]**a)/(D[i][j]**b)
    # находим вероятности перехода для каждой вершины
    for j, val in enumerate(m):
        if val == 0:
            P[j] = ((tay[i][j]**a)/(D[i][j]**b))*100 / Pz
            v.append(j)
    # добавляем слуйчаную составляющую в 50% слуйчаев
    if np.random.choice([0, 1, 2, 4]) != 1:
        # из вершины с максимальной вероятностью отнимаем случайное значение 
        # и прибавляем к вероятности слуйчаной вершины
        ran_com = np.random.randint(0, int(P.max()))
        P[P.argmax()] -= ran_com
        P[np.random.choice(v)] += ran_com
    return P.argmax()
#-------------------------------------------------------------------------------------------------
# расчет ферромонов
def pheromones(F, Q, L, T, n):
    for i in range(n):
        if i == n-1:
            F[T[i]][T[0]] += Q/L
        else:
            F[T[i]][T[i+1]] += Q/L
    return F
#-------------------------------------------------------------------------------------------------
# создание и равномерное распределение муравьев по вершинам
def making_ants(N, n):
    M = np.zeros((N, n))
    i = 0
    j = n
    for k in range(int(N/n)):
        M[i:j][np.diag_indices_from(M[i:j])] = 1
        i+=n
        j+=n
    return M
#-------------------------------------------------------------------------------------------------
# муравьиный алгоритм
def ant_algorithm(D):
    # константные параметры (для лучшей работы алгоритма будем искать наилучшую комбинацию параметров)
    A = [0.8, 1]
    B = [0.8, 1]
    # колличесво вершин
    n = len(D)
    # задаем начальное решение для дальнейшего сравнения
    T_best = np.arange(n)
    # стоимость начального решения
    L_best =cost(T_best, D, n)
    # перебор комбинаций параметров
    for b in B:
        print(b)
        for a in A:
            # порядок цены оптимального решения
            Q = 100
            # интенсивность испарения
            p = 0
            # колличесво муравьев (кратно числу вершин)
            N = n*4
            # колличесво эпох
            era = 100
            # задаем начальную концентрацию феромона, значеними от 1 до 2
            tay = np.random.randint(10, 20, size=(n, n))/10
            tay[np.diag_indices_from(tay)] = 0
            # t - номер эпохи
            t = 0
            # цикл по эпохам
            while(t < era):
                # создаем муравьев, муравьи равномерно расположены в вершинах
                # 1 - вершина в котором находиться муравей
                # 0 - вершина в котором муравей не был
                # -1 - вершина в котором муравей был
                M = making_ants(N, n)
                # матрица решений
                T = np.zeros((N,n), dtype=int)
                # матрица стоимости решений
                L = np.zeros(N, dtype=int)
                # матрица оставляемых ферромонов для каждого участка
                F = np.zeros((n,n))
                # цикл по муравьям
                for ind, m in enumerate(M):
                    # фиксируем первое положение муравья
                    T[ind][0] = np.where(m == 1)[0][0]
                    # цикл по вершинам
                    for step in range(1, n):
                        # i - текцущее местонахождения муравья
                        i = np.where(m == 1)[0][0]
                        # j - следующая вершина пути
                        j = peak_search(m, a, b, tay, D, i)
                        # фиксируем перемещение
                        M[ind][i] = -1
                        M[ind][j] = 1
                        T[ind][step] = j
                    # стоимость пути текущего муравья
                    L[ind] = cost(T[ind], D, n)
                    # феромоны, оставляемые муравьем
                    F = pheromones(F, Q, L[ind], T[ind], n)
                # обновляем феромон
                tay = tay*(1-p) + F
                # выбираем лучшее решение из полученных
                if L_best > L.min():
                    print(a, b, L.min())
                    L_best = L.min()
                    T_best = T[L.argmin()]
                # увеличивыем эпоху
                t += 1
    return L_best

# ввод матрицы
a = list(map(int, input().split()))
D = np.zeros((len(a), len(a)))
D[0] += a
for i in range(1, len(a)):
    a = list(map(int, input().split()))
    D[i] += a
print(ant_algorithm(D))